<img src="./images/kisti_logo.jpg" width="100"> <img src="./images/unist_logo.jpg" width="100"> <img src="./images/kisti_mas.png" width="90"> <img src="./images/unist_mas.jpg" width="70">

# **2025년 겨울방학 AI 팀프로젝트 슈퍼컴퓨팅 청소년 캠프**

---
### **Edge Device를 이용한 AI 실습**
### **2. 인공신경망을 이용한 이미지 훈련 및 검증**
---

**<span style="color:red">과제 시 아래 셀과 [model_definitions.py](./model_definitions.py) 수정 가능</span>**

In [ ]:
# 훈련에 이용할 데이터 설정
data_dir = "./classification_sample/thumbs_A"

# 하이퍼파라미터 설정
batch_size = 32
learning_rate = 0.001
epochs = 10

# 출력 모델 이름 설정: 모델 제출 시 "model_{조번호}.pth" 로 제출, (예:1조=model_1.pth)
model_name = 'model_full.pth'

---
### 네크워크에 입력할 문제 정보 확인

In [ ]:
import os

#--------------------------------------
# 이미지 크기 설정
image_width = 320   # 고정!!, 수정 금지
image_height = 240  # 고정!!, 수정 금지

# 데이터 디렉토리 유효성 검사
print('현재 위치:',os.getcwd())  # 현재 작업 디렉토리 출력

if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Data directory '{data_dir}' does not exist.")

# 이미지 크기와 채널 설정
image_RGB = 3
input_size = image_width * image_height * image_RGB

# 출력 확인
print(f"입력 크기: {input_size}")

---
### 데이터 전처리
- 보다 정확한 모델을 만들기 위해 이미지를 전처리 하여 효율적인 모델을 만들 수 있도록 이미지 처리

In [ ]:
import torchvision.transforms as transforms

# 데이터 전처리 및 증강 설정
use_augmentation = True  # 데이터 증강 여부 설정

transform = transforms.Compose([
    # 데이터 증강 (필요한 경우 활성화)
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2) if use_augmentation else transforms.Lambda(lambda x: x),
    # 이미지 크기 조정
    transforms.Resize((image_width, image_height)),
    # 텐서 변환
    transforms.ToTensor(),
    # 정규화 (ImageNet 사전 학습 모델과 호환)
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

---
### 이미지 데이터 입력 및 처리

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets

# 데이터셋 로드
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print('전체 이미지 갯수:',len(dataset))

# 데이터 디렉토리 내 실제 카테고리 확인
actual_category = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
print(f"폴더 '{data_dir}' 안에 있는 실제 categories name: {actual_category}")

# 데이터셋 크기와 분할 비율 설정
train_ratio = 0.8  # 훈련 데이터 비율
torch.manual_seed(42)  # 랜덤 시드 설정
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# 데이터셋 확인
print(f"훈련 데이터 셋 크기: {len(train_dataset)}")
print(f"검증 데이터 셋 크기: {len(val_dataset)}")

---
### 샘플 이미지 확인을 통한 데이터 확인

In [ ]:
from PIL import Image
from IPython.display import display

print("클래스 당 하나의 샘플 이미지 출력:")

for category, class_idx in dataset.class_to_idx.items():
    # 해당 클래스의 인덱스 필터링
    indices = [i for i, (_, label) in enumerate(dataset.samples) if label == class_idx]
    
    if indices:
        # 첫 번째 샘플 경로 가져오기
        sample_path, label = dataset.samples[indices[0]]
        try:
            # PIL을 사용해 이미지를 열고 표시
            img = Image.open(sample_path)
            print(f"카테고리: {category}")
            display(img)
        except Exception as e:
            print(f"이미지 로드 실패: {sample_path}, 오류: {e}")
    else:
        print(f"클래스 '{category}'와 관련된 샘플 없음!")

---
### 학습 및 검증 과정 설정

In [ ]:
# 학습 함수 진행 과정
def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()  # 학습 모드로 설정
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 순전파 (Forward)
        if images.size(0) == 1:
            model.eval()
            outputs = model(images)
            model.train()
        else:
            outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 (Backward)
        optimizer.zero_grad()  # 이전 그래디언트 초기화
        loss.backward()        # 그래디언트 계산

        # Gradient clipping (optional)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # gradient 증가 제한으로 급변동 방지

        optimizer.step()       # 모델 업데이트

        # 손실과 정확도 계산
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    avg_loss = total_loss / len(train_loader)
    return avg_loss, accuracy

# 검증 함수 진행 과정
def validate(model, val_loader, criterion):
    model.eval()     # 평가 모드로 설정
    total_loss = 0   # 정확도 확인
    correct = 0      
    total = 0

    with torch.no_grad():  # 그래디언트 계산 비활성화
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # 순전파 (Forward)
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 손실과 정확도 계산
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    avg_loss = total_loss / len(val_loader)
    return avg_loss, accuracy

---
### 모델 설정 [(model_definitions.py)](./model_definitions.py)

In [ ]:
# 모델 import
from model_definitions import SimpleNN
import torch.nn as nn
import torch.optim as optim
import torchvision

num_categories = len(actual_category)

# 모델 초기화
model = SimpleNN(input_size=image_width*image_height*image_RGB, num_classes=num_categories)
                           
# 가중치 초기화 함수 정의
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

# 모델 가중치 초기화
model.apply(initialize_weights)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습률 스케줄러 추가 (선택)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 하드웨어 메시지 출력
if device.type == "cuda":
    print("cuda 이용")
else:
    print("cpu 이용")
    
# 모델 요약 출력
print(model)
print(f"Total parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

---
### **훈련시작**

In [ ]:
# 학습 및 검증 루프
# epochs 수 만큼 훈현 반복
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    print(f'Epoch [{epoch+1}/{epochs}] '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}% | '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.2f}%')
    print("-" * 30)

---
### 검증 용 샘플 확인

In [ ]:
from IPython.display import display
from PIL import Image

# Validation 단계에서 샘플 출력 추가
sample_display_count = 3  # 출력할 샘플 수

# 샘플 검증 함수 정의
def validate_samples_with_accuracy(model, val_loader, categories, sample_display_count=5):
    """
    모델을 사용하여 검증 데이터의 샘플을 출력하고 예측 결과와 정확도를 표시.

    Args:
        model (torch.nn.Module): 학습된 PyTorch 모델
        val_loader (DataLoader): 검증 데이터 로더
        class_names (list): 클래스 이름 리스트
        sample_display_count (int): 출력할 샘플 수
    """
    model.eval()          # 모델을 평가 모드로 전환
    sample_displayed = 0  # 출력된 샘플 수를 추적

    # Reverse transformation to display original image
    reverse_transform = transforms.Compose([
        # Normalize를 역으로 적용하여 원본 이미지 복원
        transforms.Normalize(mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225], 
                             std=[1 / 0.229, 1 / 0.224, 1 / 0.225]),
        # 텐서를 PIL 이미지로 변환
        transforms.ToPILImage()
    ])

    with torch.no_grad():  # 그래디언트 계산 비활성화 (평가 모드에 적합)
        for images, labels in val_loader:
            # 검증 데이터 이미지와 라벨을 장치로 이동
            images, labels = images.to(device), labels.to(device)

            # 모델 예측 수행
            outputs = model(images)                                      # 모델의 forward 메서드 호출
            probabilities = torch.nn.functional.softmax(outputs, dim=1)  # 각 클래스에 대한 확률 계산
            _, predicted = torch.max(outputs, 1)                         # 가장 높은 확률의 클래스 인덱스 추출

            for i in range(images.size(0)):                              # 현재 배치의 모든 샘플을 순회
                if sample_displayed >= sample_display_count:             # 출력할 샘플 수를 초과하면 종료
                    return

                # 원본 이미지 복원
                original_image = reverse_transform(images[i].cpu())      # CPU로 이동 후 변환

                # 원본 크기로 이미지를 유지 (height, width 순으로 전달 필요)
                original_image = original_image.resize((images.size(2), images.size(3)))

                # 예측 결과와 확률 계산
                pred_class = categories[predicted[i].item()]  # 예측된 클래스 이름
                true_class = categories[labels[i].item()]     # 실제 클래스 이름
                pred_confidence = probabilities[i][predicted[i].item()] * 100  # 예측값 확률 (%)

                # 이미지 출력 및 예측 정보 표시
                display(original_image)  # IPython.display를 통해 이미지 출력
                print(f"True: {true_class}, Pred: {pred_class} ({pred_confidence:.2f}%)")
                if true_class != pred_class:
                    print("Test Fail")

                sample_displayed += 1  # 출력된 샘플 수 증가

# 검증 데이터에서 샘플 출력
validate_samples_with_accuracy(model, val_loader, actual_category, sample_display_count)

---
### **모델 저장**|

In [ ]:
# 모델 저장
torch.save(model,model_name)